In [2]:
import pandas as pd
import re           #regular expressions
from unicodedata import normalize       # para quitar tildes

In [29]:
guille = "000"
noms = guille.split('.')
noms[0]

'000'

In [30]:
df_norm = pd.read_excel("data_sipra.xlsx")
df_norm['Departamento']=df_norm['Departamento'].str.upper()
df_norm['Cultivo']=df_norm['Cultivo'].str.upper()
# Quito tildes
df_norm["Departamento"] = df_norm["Departamento"].apply(lambda x: re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", x), 0, re.I))

df_norm

def changenumber(df):
    numbers = []
    for number in df:
        number = str(number)
        if "." in number:
            noms = number.split(".")
            if len(noms)==1:
                numbers.append(number)
                nom = noms[0]
            elif len(noms)==2:
                if len(noms[1])<2:
                    noms[1]=noms[1]+"00"
                elif len(noms[1])==2:
                    noms[1]=noms[1]+"0"
                if len(noms[1])>3:
                    numero=noms[1]
                    noms[1]=numero[0:3]
                noms = noms[0]+noms[1]
            elif len(noms)==3:
                if len(noms[1])<2:
                    noms[1]=noms[1]+"00"
                elif len(noms[1])==2:
                    noms[1]=noms[1]+"0"
                if len(noms[2])<2:
                    noms[2]=noms[2]+"00"
                elif len(noms[2])==2:
                    noms[2]=noms[2]+"0"
                noms = noms[0]+noms[1]+noms[2]
            else:
                noms = 0
        else:
            noms = number
        numbers.append(noms)
    # print(numbers)
    return(numbers)

df_norm["Área departamento [ha]"]=changenumber(df_norm["Área departamento [ha]"])
df_norm["Área departamento [ha]"]=df_norm["Área departamento [ha]"].astype('int')
df_norm["Alta [ha]"]= changenumber(df_norm["Alta [ha]"])
df_norm["Alta [ha]"]=df_norm["Alta [ha]"].astype('int')
df_norm["Media [ha]"]= changenumber(df_norm["Media [ha]"])
df_norm["Media [ha]"]=df_norm["Media [ha]"].astype('int')
df_norm["Baja [ha]"]= changenumber(df_norm["Baja [ha]"]) 
df_norm["Baja [ha]"]=df_norm["Baja [ha]"].astype('int')
df_norm["Total aptitud [ha]"]= changenumber(df_norm["Total aptitud [ha]"]) 
df_norm["Total aptitud [ha]"]= df_norm["Total aptitud [ha]"].astype('int')
df_norm

,Unnamed: 0,Cultivo,Departamento,Alta [ha],Media [ha],Baja [ha],Muy baja [ha],Total aptitud [ha],Área departamento [ha]
0,0,AGUACATE,ANTIOQUIA,275838,251051,167469,0,694358,6296299
1,1,AGUACATE,CAUCA,160441,195942,113245,0,469628,3125130
2,2,AGUACATE,BOYACA,130341,104622,126270,0,361233,2317531
3,3,AGUACATE,SANTANDER,95357,153818,80125,0,329300,3054326
4,4,AGUACATE,CUNDINAMARCA,171181,92777,30911,0,294869,2236783
...,...,...,...,...,...,...,...,...,...
859,27,PLANTACIONES FORESTALES,QUINDIO,40856,44078,12286,0,97221,193217
860,28,PLANTACIONES FORESTALES,VAUPES,0,0,50049,0,50049,5343179
861,29,PLANTACIONES FORESTALES,AMAZONAS,0,4784,18130,0,22914,10903686
862,30,PLANTACIONES FORESTALES,GUAINIA,0,1049,15956,0,17005,7140386


In [31]:
# Coger los nombres de departamentos del Json
import json
with open('datos/departamentos.json') as f:
    counties = json.load(f)

Departamentos = []
geocol = counties.copy()
#Sacar una lista de los departamentos llamada "Departamentos"
for man in geocol['features']:
    Departamentos.append(man['properties']['NOMBRE_DPT'])
    man['id']=man['properties']['NOMBRE_DPT']

# Departamentos
# # Por cada departamento del Json
for depto in Departamentos:
    # si está ya en el data set, no hacer nada, si no está hacer lo siguiente
    if df_norm['Departamento'].str.contains(depto).any() == False:
        # Revisar cantidad de palabras
        words = depto.split()
        # Si hay una sola palabra, coger el 60 % de esa palabra y buscarlo
        if len(words) == 1:
            df_norm.loc[df_norm.Departamento.str.find(depto[:int(len(depto)*0.6)])>=0, 'Departamento'] = depto
        # Si hay entre 2 o 3 palabras por lo general se encuentra casi la misma palabra (80%)
        elif len(words) > 1 and len(words) <= 3:
            df_norm.loc[df_norm.Departamento.str.find(depto[:int(len(depto)*0.8)])>=0, 'Departamento'] = depto
        # si hay mas de 4 palabras, el unico registro es san andres
        elif len(words) > 4:
            df_norm.loc[df_norm.Departamento.str.find("SAN ANDRES")>=0, 'Departamento'] = depto

In [32]:
# Abrir dictionary ADR
dictionary_ADR = pd.read_csv("datos/cadenas_productivas_ADR.csv", sep=';', engine='python')
dictionary_ADR= dictionary_ADR[dictionary_ADR['NIVEL']==3]
dictionary_ADR["CADENA_PRODUCTIVA_ADR"] = dictionary_ADR["CADENA_PRODUCTIVA_ADR"].apply(lambda x: re.sub(r"([^n\u0300-\u036f]|n(?!\u0303(?![\u0300-\u036f])))[\u0300-\u036f]+", r"\1", normalize( "NFD", x), 0, re.I))

,Unnamed: 0,Cultivo,Departamento,Alta [ha],Media [ha],Baja [ha],Muy baja [ha],Total aptitud [ha],Área departamento [ha]
0,0,AGUACATE,ANTIOQUIA,275838,251051,167469,0,694358,6296299
1,1,AGUACATE,CAUCA,160441,195942,113245,0,469628,3125130
2,2,AGUACATE,BOYACA,130341,104622,126270,0,361233,2317531
3,3,AGUACATE,SANTANDER,95357,153818,80125,0,329300,3054326
4,4,AGUACATE,CUNDINAMARCA,171181,92777,30911,0,294869,2236783
...,...,...,...,...,...,...,...,...,...
859,27,PLANTACIONES FORESTALES,QUINDIO,40856,44078,12286,0,97221,193217
860,28,PLANTACIONES FORESTALES,VAUPES,0,0,50049,0,50049,5343179
861,29,PLANTACIONES FORESTALES,AMAZONAS,0,4784,18130,0,22914,10903686
862,30,PLANTACIONES FORESTALES,GUAINIA,0,1049,15956,0,17005,7140386


In [33]:
cadenas_dictionary = dictionary_ADR["CADENA_PRODUCTIVA_ADR"].tolist()

df = df_norm[df_norm['Cultivo']!="PAPA "]
for i in range(len(cadenas_dictionary)):
    row = cadenas_dictionary[i]
    # Comparo cada registro del diccionario de ADR con los registros del IPC, además se debe de tener en cuenta que deben ser palabras sueltas
    #df.loc[((df['Cultivo'].str.contains(row))|(df['Cultivo'].str.contains(row+"S"))), 'Cultivo'] = row
    df.loc[((df['Cultivo'].str.contains(rf"\b({row})\b"))|(df['Cultivo'].str.contains(rf"\b({row}+S)\b"))), 'Cultivo'] = row
#quito nans (los que no se encontraron)
# df=df.dropna()
df

,Unnamed: 0,Cultivo,Departamento,Alta [ha],Media [ha],Baja [ha],Muy baja [ha],Total aptitud [ha],Área departamento [ha]
0,0,AGUACATE,ANTIOQUIA,275838,251051,167469,0,694358,6296299
1,1,AGUACATE,CAUCA,160441,195942,113245,0,469628,3125130
2,2,AGUACATE,BOYACA,130341,104622,126270,0,361233,2317531
3,3,AGUACATE,SANTANDER,95357,153818,80125,0,329300,3054326
4,4,AGUACATE,CUNDINAMARCA,171181,92777,30911,0,294869,2236783
...,...,...,...,...,...,...,...,...,...
859,27,PLANTACIONES FORESTALES,QUINDIO,40856,44078,12286,0,97221,193217
860,28,PLANTACIONES FORESTALES,VAUPES,0,0,50049,0,50049,5343179
861,29,PLANTACIONES FORESTALES,AMAZONAS,0,4784,18130,0,22914,10903686
862,30,PLANTACIONES FORESTALES,GUAINIA,0,1049,15956,0,17005,7140386


In [34]:
cultivos = []
for cultivo in df['Cultivo']:
    if not cultivo in cadenas_dictionary:
        cultivo = "NO"
    cultivos.append(cultivo)
df['Cultivo']=cultivos

In [35]:
df

,Unnamed: 0,Cultivo,Departamento,Alta [ha],Media [ha],Baja [ha],Muy baja [ha],Total aptitud [ha],Área departamento [ha]
0,0,AGUACATE,ANTIOQUIA,275838,251051,167469,0,694358,6296299
1,1,AGUACATE,CAUCA,160441,195942,113245,0,469628,3125130
2,2,AGUACATE,BOYACA,130341,104622,126270,0,361233,2317531
3,3,AGUACATE,SANTANDER,95357,153818,80125,0,329300,3054326
4,4,AGUACATE,CUNDINAMARCA,171181,92777,30911,0,294869,2236783
...,...,...,...,...,...,...,...,...,...
859,27,NO,QUINDIO,40856,44078,12286,0,97221,193217
860,28,NO,VAUPES,0,0,50049,0,50049,5343179
861,29,NO,AMAZONAS,0,4784,18130,0,22914,10903686
862,30,NO,GUAINIA,0,1049,15956,0,17005,7140386


In [36]:
df_normalizado=df[df['Cultivo']!="NO"]
df_normalizado

,Unnamed: 0,Cultivo,Departamento,Alta [ha],Media [ha],Baja [ha],Muy baja [ha],Total aptitud [ha],Área departamento [ha]
0,0,AGUACATE,ANTIOQUIA,275838,251051,167469,0,694358,6296299
1,1,AGUACATE,CAUCA,160441,195942,113245,0,469628,3125130
2,2,AGUACATE,BOYACA,130341,104622,126270,0,361233,2317531
3,3,AGUACATE,SANTANDER,95357,153818,80125,0,329300,3054326
4,4,AGUACATE,CUNDINAMARCA,171181,92777,30911,0,294869,2236783
...,...,...,...,...,...,...,...,...,...
507,27,PAPAYA,QUINDIO,47444,6667,0,0,54112,193217
508,28,PAPAYA,AMAZONAS,93,1182,11069,0,12344,10903686
509,29,PAPAYA,GUAINIA,0,3060,8631,0,11691,7140386
510,30,PAPAYA,VAUPES,0,0,6844,0,6844,5343179


In [37]:
df_normalizado[df_normalizado['Cultivo']=="PAPA"].groupby("Departamento").max()

,Unnamed: 0,Cultivo,Alta [ha],Media [ha],Baja [ha],Muy baja [ha],Total aptitud [ha],Área departamento [ha]
Departamento,,,,,,,,
AMAZONAS,26,PAPA,0,0,0,0,0,10903686
ANTIOQUIA,5,PAPA,4171,5376,7268,0,16815,6296299
ARAUCA,22,PAPA,0,0,0,0,0,2383135
ARCHIPIELAGO DE SAN ANDRES PROVIDENCIA Y SANTA CATALINA,25,PAPA,0,0,0,0,0,4972
ATLANTICO,24,PAPA,0,0,0,0,0,331159
BOLIVAR,20,PAPA,0,0,0,0,0,2665496
BOYACA,2,PAPA,21655,19214,18500,0,59369,2317531
CALDAS,9,PAPA,426,656,1925,0,3007,743890
CAQUETA,19,PAPA,0,0,0,0,0,9010823


In [38]:
df_normalizado.to_csv("data_sipra_normalizado.csv")
df_normalizado.to_excel("data_sipra_normalizado.xlsx")

In [43]:
cadenas_dictionary

['CAFE',
 'CEBOLLA',
 'CEBOLLA DE BULBO',
 'CEBOLLA DE RAMA',
 'CEBOLLIN',
 'CADUCIFOLIOS',
 'CANYARANA',
 'PLANTULAS',
 'EUCALIPTO BABY BLUE',
 'ROSA',
 'CLAVEL',
 'ORQUIDEA',
 'CRISANTEMOS',
 'ANTURIO',
 'ASTROMELIA',
 'CARTUCHO-ASTROMELIA',
 'CORDELINE CINTA',
 'FITOSPORUM',
 'FLOR DE SAFARI',
 'GERBERAS',
 'GLADIOLO',
 'HELECHO',
 'HELECHO CUERO',
 'HELICONIA',
 'HORTENSIA',
 'LIMONIUM',
 'MORERA',
 'PALMA ROBELINA',
 'POMPON Y CRISANTEMO',
 'PROTEA',
 'STATICE',
 'TREE FERN',
 'PAPA',
 'PAPA CHINA',
 'PAPA CRIOLLA',
 'ULLUCO',
 'APIO',
 'ARRACACHA',
 'BATATA',
 'BORE',
 'CHONQUE',
 'MALANGA',
 'ÑAME',
 'ÑAME DIAMANTE',
 'ÑAME ESPINO',
 'YACON',
 'YUCA',
 'YUCA INDUSTRIAL',
 'TOMATE',
 'TOMATE INVERNADERO',
 'AJO',
 'PUERRO',
 'BROCOLI',
 'COL',
 'COLIFLOR',
 'REPOLLO',
 'ACHICORIA',
 'LECHUGA',
 'ACELGA',
 'NABO',
 'ZANAHORIA',
 'RABANO',
 'REMOLACHA',
 'PEPINO COHOMBRO',
 'PEPINO GUISO',
 'ARVEJA',
 'CHACHAFRUTO',
 'FRIJOL',
 'FRIJOL ARBUSTIVO',
 'FRIJOL LADERA',
 'FRIJOL PLAN

In [42]:
df_normalizado.groupby('Cultivo').mean()

,Unnamed: 0,Alta [ha],Media [ha],Baja [ha],Muy baja [ha],Total aptitud [ha],Área departamento [ha]
Cultivo,,,,,,,
AGUACATE,15.5,37469.90625,37849.59375,24546.28125,0.00000,99865.75000,3.559791e+06
ARROZ,15.5,51669.21875,103399.75000,249975.40625,0.00000,405044.43750,3.559791e+06
BANANO,15.5,21987.15625,41494.84375,95787.81250,0.00000,159269.87500,3.559791e+06
CACAO,15.5,227427.87500,198551.21875,97480.84375,0.00000,523460.00000,3.559791e+06
CAUCHO,15.5,246323.43750,180070.62500,81332.56250,0.00000,509381.25000,3.559791e+06
CEBOLLA,15.5,143535.62500,211718.68750,170224.87500,0.00000,525479.18750,3.559791e+06
FRESA,15.5,34790.31250,35271.84375,28026.87500,0.00000,98089.12500,3.559791e+06
MANGO,15.5,123463.15625,160779.21875,180014.87500,0.00000,464257.31250,3.559791e+06
PALMA DE ACEITE,15.5,161709.40625,338452.21875,89432.50000,134028.96875,723623.09375,3.425762e+06
